<a href="https://colab.research.google.com/github/dvschultz/lucid-sonic-dreams/blob/main/Lucid_Sonic_Dreams_PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lucid Sonic Dreams

This is a modified version of [Lucid Sonic Dreams](https://github.com/mikaelalafriz/lucid-sonic-dreams) (a modification of the [PyTorch version here](https://github.com/NotNANtoN/lucid-sonic-dreams).) It allows you to modify the output of a StyleGAN-ADA-PyTorch model with an audio signal.

Much of the documentation included here comes directly from [the LSD notebook](https://colab.research.google.com/drive/1Y5i50xSFIuN3V4Md8TB30_GOAtts7RQD). Note that I’ve modified the repo used here so that notebook and this one are not 1:1 compatible.

##Setup

In [ ]:
!nvidia-smi -L

Once you’ve restarted jump down here and continue the setup.

In [ ]:
%cd /content/
!git clone https://github.com/dvschultz/lucid-sonic-dreams
!pip install pygit2 mega.py ninja
%cd /content/lucid-sonic-dreams

It’s optional if you want to sync your files to Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Download a StyleGAN model

You can either download a model, or use the path to your own trained model

In [ ]:
from lucidsonicdreams import show_styles 

# Show valid default style names. 
show_styles()

In [ ]:
# trees, padded (512)
!gdown --id 12WS9F0S4yzZQERy4dsN-3ZoEzHPAViMP
# drawn minerals (1024)
#!gdown --id 1YKkozuo2Ik_SGMwh-mFH0I9ZzCN2_gTn

## Run

1. Upload an mp3 to Colab.
2. Edit the paths below to point to your mp3, your .pkl file, and then an output path.
3. Run the cell.

### So, how does it work? 

1. First, a batch of input vectors corresponding to output images is initialized. Linear interpolations between these vectors are produced, serving as the "base" vectors.
2. Three components react to the audio: **Pulse**, **Motion**, and **Class**. These modify the "base" vectors accordingly.

  *   **Pulse**, quite literally, refers to how the visuals "pulse" to the beat of the music. It is set to react to the audio's percussive elements by default. 
  *   **Motion** refers to how the visuals are "pushed forward" or "sped up" by the music, and is set to react to the audio's harmonic elements by default. 
3. Finally, additional effects - such as contrast and flash - are added to the video. These are set to react to the audio's percussive elements by default.

In [ ]:
from lucidsonicdreams import LucidSonicDream

L = LucidSonicDream(song = '/content/1467_Empty_Love_feat__Ed_Thomas.mp3', #mp3 here
                    style = 'abstract art') #pkl path here

L.hallucinate(file_name = '/content/basic-lsd.mp4', #output path here
              start = 0,
              duration=10,  
              speed_fpm = 12,
              pulse_percussive = True,
              pulse_harmonic= False,
              pulse_react = 1.0,
              motion_harmonic = True,
              motion_percussive = False,
              random_seed=1,
              ) 

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('1hhIare4VaA')

## Source Separation of Audio Track

While an mp3 will work fine with Lucid Sonic Dreams, it will work better if you separate your song into individual tracks. We’ll use a tool called demucs to turn our single mp3 into individual tracks: drums, bass, vocals, and "other" (melody).

In [ ]:
!pip install --no-deps torchaudio==0.9.1 julius diffq
!pip install tqdm lameenc
!pip install --no-deps demucs

In [ ]:
!python -m demucs.separate -h

There are a number of models you can use using the `-n` argument. A list is available [here](https://github.com/facebookresearch/demucs#separating-tracks).

In [ ]:
!python -m demucs.separate \
-o /content/demucs_separated/ \
-n demucs_extra \
--mp3 --mp3-bitrate=320 \
/content/1467_Empty_Love_feat__Ed_Thomas.mp3

There are even more examples and documentation in [this notebook](https://colab.research.google.com/drive/1Y5i50xSFIuN3V4Md8TB30_GOAtts7RQD).

(Note: your PyTorch model will not work with that notebook, but you can cut and paste code from there into here to run it.)

### Reactive Motion
* **speed_fpm** (*Default: 12*) - FPM stands for "Frames per Minute". This determines how many images are initialized - the more there are, the faster the visuals morph. If **speed_fpm = 0**, then only one image is initialized, and that single image reacts to the audio. In this case, there will be no motion during silent parts of the audio.
*   **motion_react** (*Default: 0.5*) - The "strength" of the motion. It is recommended to keep this between 0 and 2.

In [ ]:
from lucidsonicdreams import LucidSonicDream
from PIL import ImageEnhance

L = LucidSonicDream(song = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/vocals.mp3',
                    pulse_audio = None,
                    motion_audio = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/vocals.mp3',
                    contrast_audio = None,
                    flash_audio = None,
                    style = 'abstract art')

L.hallucinate('sourceseparated-lsd_motion-vocals.mp4',
              start = 0, 
              duration = 10, 
              speed_fpm = 12,
              motion_react = 1.0,
              truncation = 1.0,
              pulse_react = 0.0,
              pulse_percussive = False,
              contrast_percussive = False,
              fps = 24, 
              flash_percussive = False,
              )

In [ ]:
YouTubeVideo('VZKDNq8IVLs')

## Reactive Pulse

*   **pulse_react** (*Default: 0.5*) - The "strength" of the pulse. It is recommended to keep this between 0 and 2.

In [ ]:
from lucidsonicdreams import LucidSonicDream

L = LucidSonicDream(song = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/drums.mp3',
                    pulse_audio = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/drums.mp3',
                    motion_audio = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/vocals.mp3',
                    contrast_audio = None,
                    flash_audio = None,
                    style = 'abstract art')

L.hallucinate('/conent/sourceseparated-lsd_pulse-drums-2.0.mp4',
              start = 0, 
              duration = 10, 
              speed_fpm = 0, # 12
              pulse_react = 1.5, # 0-2.0
              motion_react = 0.0,
              contrast_percussive = False,
              fps = 24, 
              flash_percussive = False,
              )

In [ ]:
YouTubeVideo('knOF8Hv3Ogk')

Once we find a setting we like, we can combine our previous motion settings with the new pulse settings.

If you want to combine two of the separated tracks you can use ffmmpeg:

In [ ]:
!ffmpeg -i /content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/drums.mp3 \
-i /content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/vocals.mp3 \
-filter_complex amix=inputs=2:duration=longest \
/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/vocals+drums.mp3

In [ ]:
from lucidsonicdreams import LucidSonicDream

L = LucidSonicDream(song = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/vocals+drums.mp3',
                    pulse_audio = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/drums.mp3',
                    motion_audio = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/vocals.mp3',
                    contrast_audio = None,
                    flash_audio = None,
                    style = 'abstract art')

L.hallucinate('sourceseparated-lsd_vocals+drums.mp4',
              start = 0, 
              duration = 45, 
              speed_fpm = 12, # 12
              pulse_react = 1.5, # 0-2.0
              motion_react = 1.0,
              contrast_percussive = False,
              fps = 24, 
              flash_percussive = False
              )

In [ ]:
YouTubeVideo('MkFH8Ck5Xts')

### Final Touches

*   **contrast_strength** (*Default: 0.5*) - Strength of default contrast effect. It is recommended to keep this between 0 and 1.
*   **flash_strength** (*0.5*) - Strength of default flash effect. It is recommended to keep these between 0 and 1.

Again, let’s isolate the variables to see exactly what happens.

In [ ]:
from lucidsonicdreams import LucidSonicDream

L = LucidSonicDream(song = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/drums.mp3',
                    pulse_audio = None,
                    motion_audio = None,
                    contrast_audio = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/drums.mp3',
                    flash_audio = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/drums.mp3',
                    style = 'abstract art')

L.hallucinate('sourceseparated-lsd_contrast.mp4',
              start = 0, 
              duration = 45, 
              speed_fpm = 0,
              pulse_react = 0,
              motion_react = 0.0,
              contrast_strength = 1.0,
              fps = 24, 
              flash_strength = 0.0,
              truncation = 0.7,
              random_seed = 0
              )

In [ ]:
YouTubeVideo('BOujZ4ruGY0')

In [ ]:
from lucidsonicdreams import LucidSonicDream

L = LucidSonicDream(song = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/drums.mp3',
                    pulse_audio = None,
                    motion_audio = None,
                    contrast_audio = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/drums.mp3',
                    flash_audio = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/drums.mp3',
                    style = 'abstract art')

L.hallucinate('sourceseparated-lsd_effects.mp4',
              start = 0, 
              duration = 45, 
              speed_fpm = 0,
              pulse_react = 0,
              motion_react = 0.0,
              contrast_strength = 1.0,
              fps = 24, 
              flash_strength = 1.0,
              truncation = 0.7,
              random_seed = 0
              )

In [ ]:
YouTubeVideo('XThn9gna1t0')

In [ ]:
from lucidsonicdreams import LucidSonicDream

L = LucidSonicDream(song = '/content/1467_Empty_Love_feat__Ed_Thomas.mp3',
                    pulse_audio = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/drums.mp3',
                    motion_audio = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/vocals.mp3',
                    contrast_audio = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/drums.mp3',
                    flash_audio = '/content/demucs_separated/demucs_extra/1467_Empty_Love_feat__Ed_Thomas/drums.mp3',
                    style = 'abstract art')

L.hallucinate('sourceseparated-lsd_final.mp4',
              start = 0, 
              duration = 45, 
              speed_fpm = 12,
              pulse_react = 1.5,
              motion_react = 0.75,
              contrast_strength = 0.75,
              fps = 24, 
              flash_strength = 0.5,
              truncation = 0.7
              )

In [ ]:
YouTubeVideo('3N61h9RMbcU')